In [5]:
# -*- coding: utf-8 -*-
"""
Deming regression: Sentinel-3 (reference) vs all Sentinel-2 chl-a versions.

Model per pair (linear space):
    y = a + b x
where x = S3, y = S2, fitted with Deming regression (orthogonal errors).
For log space, we regress log10(S2) on log10(S3); the mapping is then
    S2_cal = 10^a * (S2_raw)^b  if you calibrate S2 to match S3.

Outputs:
  - s2_vs_s3_regression_summary.csv
  - plots/s2_vs_s3_<S2COL>.png
"""

import os
import re
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
from matplotlib.offsetbox import AnchoredText
from typing import List, Optional, Tuple

In [6]:
# -------------------- CONFIG --------------------
CSV_PATH = "/home/data/ocean-colour/s2_s3_sampling_wolines_300m.csv"

In [7]:
# If auto-detection fails, set these explicitly:
S3_COL_NAME: Optional[str] = None     # e.g. "S3"
S2_INCLUDE_REGEX: Optional[str] = None  # e.g. r"^S2_.*"

# Work in log10 space? (recommended for chl-a)
LOG10_TRANSFORM = False

# Simple outlier removal (after transform)
REMOVE_OUTLIERS = True
OUTLIER_METHOD = "iqr"   # "iqr" or "zscore"
IQR_K = 1.5
ZSCORE_THRESH = 4.0

# Deming regression variance ratio λ = Var(error_y)/Var(error_x)
# Here x=S3, y=S2. If unknown, start with 1.0
LAMBDA_VAR_RATIO = 1.0

# Bootstrap for CIs
N_BOOT = 1000
BOOT_SEED = 0
CI_LO = 2.5
CI_HI = 97.5

# Plot settings
PLOT_DIR = "plots"
FIGSIZE = (6, 5)
MARKER_SIZE = 15
ALPHA = 0.6
# ------------------------------------------------

def autodetect_columns(df: pd.DataFrame) -> Tuple[str, List[str]]:
    cols = list(df.columns)
    s3_candidates = [c for c in cols if re.search(r"(?:^|[^a-zA-Z0-9])s3(?:[^a-zA-Z0-9]|$)", c, flags=re.I)
                     or re.search(r"sentinel[-_\s]*3", c, flags=re.I)]
    if not s3_candidates:
        s3_candidates = [c for c in cols if re.search(r"(?i)S3.*(chl|chla|chlor|a)\b", c)]
    if not s3_candidates:
        numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
        if len(numeric_cols) == 1:
            s3_guess = numeric_cols[0]
        else:
            raise ValueError("Could not auto-detect an S3 column. Set S3_COL_NAME explicitly.")
    else:
        s3_guess = s3_candidates[0]

    s2_candidates = [c for c in cols if (
        re.search(r"(?:^|[^a-zA-Z0-9])s2(?:[^a-zA-Z0-9]|$)", c, flags=re.I) or
        re.search(r"sentinel[-_\s]*2", c, flags=re.I)
    )]
    s2_candidates = [c for c in s2_candidates if pd.api.types.is_numeric_dtype(df[c])]

    if S2_INCLUDE_REGEX:
        s2_candidates = [c for c in cols if re.search(S2_INCLUDE_REGEX, c, flags=re.I)]
        s2_candidates = [c for c in s2_candidates if pd.api.types.is_numeric_dtype(df[c])]

    if not s2_candidates:
        numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
        s2_candidates = [c for c in numeric_cols if c != s3_guess]

    if not s2_candidates:
        raise ValueError("No S2 columns found. Consider setting S2_INCLUDE_REGEX.")

    return s3_guess, s2_candidates

def apply_transform(x: np.ndarray) -> np.ndarray:
    if not LOG10_TRANSFORM:
        return x.astype(float)
    x = x.astype(float)
    x = np.where(x > 0, x, np.nan)   # log valid only for positive
    return np.log10(x)

def remove_outliers_xy(x: np.ndarray, y: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
    if not REMOVE_OUTLIERS:
        return x, y
    xv = x.copy()
    yv = y.copy()
    finite = np.isfinite(xv) & np.isfinite(yv)
    xv = xv[finite]; yv = yv[finite]

    if OUTLIER_METHOD == "iqr":
        def iqr_keep(a):
            q1, q3 = np.nanpercentile(a, [25, 75])
            iqr = q3 - q1
            lo, hi = q1 - IQR_K * iqr, q3 + IQR_K * iqr
            return (a >= lo) & (a <= hi)
        keep = iqr_keep(xv) & iqr_keep(yv)
        return xv[keep], yv[keep]

    elif OUTLIER_METHOD == "zscore":
        zx = stats.zscore(xv, nan_policy='omit')
        zy = stats.zscore(yv, nan_policy='omit')
        keep = (np.abs(zx) <= ZSCORE_THRESH) & (np.abs(zy) <= ZSCORE_THRESH)
        return xv[keep], yv[keep]

    else:
        return xv, yv

# -------------------- DEMING REGRESSION --------------------
def deming_regression(x: np.ndarray, y: np.ndarray, lam: float = 1.0):
    """
    Closed-form Deming regression for linear model y = a + b x
    lam = Var(error_y) / Var(error_x)
    Returns (a, b)
    """
    x = np.asarray(x, dtype=float)
    y = np.asarray(y, dtype=float)
    if len(x) != len(y) or len(x) < 3:
        raise ValueError("Need paired x,y with n >= 3.")

    xbar = np.nanmean(x)
    ybar = np.nanmean(y)

    X = x - xbar
    Y = y - ybar
    Sxx = np.nanmean(X*X)
    Syy = np.nanmean(Y*Y)
    Sxy = np.nanmean(X*Y)

    # Deming slope
    num = Syy - lam*Sxx + np.sqrt((Syy - lam*Sxx)**2 + 4*lam*(Sxy**2))
    den = 2*Sxy
    b = num / den
    a = ybar - b*xbar
    return float(a), float(b)

def deming_bootstrap_ci(x: np.ndarray, y: np.ndarray, lam: float = 1.0,
                        n_boot: int = 1000, ci_lo: float = 2.5, ci_hi: float = 97.5,
                        seed: Optional[int] = None):
    """
    Pairs bootstrap for Deming regression to get CIs of (a,b).
    """
    rng = np.random.default_rng(seed)
    x = np.asarray(x, dtype=float)
    y = np.asarray(y, dtype=float)
    n = len(x)
    a_hat, b_hat = deming_regression(x, y, lam)

    A = np.empty(n_boot); B = np.empty(n_boot)
    for i in range(n_boot):
        idx = rng.integers(0, n, size=n)
        xb = x[idx]; yb = y[idx]
        try:
            a, b = deming_regression(xb, yb, lam)
        except Exception:
            a, b = np.nan, np.nan
        A[i] = a; B[i] = b

    a_ci = (np.nanpercentile(A, ci_lo), np.nanpercentile(A, ci_hi))
    b_ci = (np.nanpercentile(B, ci_lo), np.nanpercentile(B, ci_hi))
    a_se = float(np.nanstd(A, ddof=1))
    b_se = float(np.nanstd(B, ddof=1))
    return dict(
        intercept=a_hat, slope=b_hat,
        intercept_se=a_se, slope_se=b_se,
        intercept_ci_lo=float(a_ci[0]), intercept_ci_hi=float(a_ci[1]),
        slope_ci_lo=float(b_ci[0]), slope_ci_hi=float(b_ci[1])
    )
# -----------------------------------------------------------

def regression_and_stats_deming(x: np.ndarray, y: np.ndarray, lam: float = 1.0,
                                n_boot: int = 1000, seed: Optional[int] = 0) -> dict:
    # paired, finite
    x = np.asarray(x, dtype=float)
    y = np.asarray(y, dtype=float)
    finite = np.isfinite(x) & np.isfinite(y)
    x = x[finite]; y = y[finite]
    n = len(x)
    if n < 3:
        return dict(n=int(n), slope=np.nan, slope_se=np.nan, slope_ci_lo=np.nan, slope_ci_hi=np.nan,
                    intercept=np.nan, intercept_se=np.nan, intercept_ci_lo=np.nan, intercept_ci_hi=np.nan,
                    r_squared=np.nan, pearson_r=np.nan, pearson_p=np.nan, spearman_rho=np.nan, spearman_p=np.nan,
                    rmse=np.nan, mae=np.nan, bias=np.nan)

    ci = deming_bootstrap_ci(x, y, lam=lam, n_boot=n_boot, seed=seed)
    a = ci["intercept"]; b = ci["slope"]

    # correlations
    pearson_r, pearson_p = stats.pearsonr(x, y) if n > 1 else (np.nan, np.nan)
    spearman_rho, spearman_p = stats.spearmanr(x, y, nan_policy='omit') if n > 1 else (np.nan, np.nan)
    r_squared = float(pearson_r**2)

    # residuals vs Deming fit
    resid = y - (a + b*x)
    rmse = float(np.sqrt(np.nanmean(resid**2)))
    mae = float(np.nanmean(np.abs(resid)))
    bias = float(np.nanmean(y - x))  # mean(S2 - S3) in the working space

    return dict(
        n=int(n),
        slope=float(b), slope_se=float(ci["slope_se"]), slope_ci_lo=float(ci["slope_ci_lo"]), slope_ci_hi=float(ci["slope_ci_hi"]),
        intercept=float(a), intercept_se=float(ci["intercept_se"]), intercept_ci_lo=float(ci["intercept_ci_lo"]), intercept_ci_hi=float(ci["intercept_ci_hi"]),
        r_squared=r_squared, pearson_r=float(pearson_r), pearson_p=float(pearson_p),
        spearman_rho=float(spearman_rho), spearman_p=float(spearman_p),
        rmse=rmse, mae=mae, bias=bias
    )

def add_stats_box(ax, stats_row: dict, loc="lower right", box_alpha=0.9):
    txt = (f"n={stats_row['n']}\n"
           f"slope={stats_row['slope']:.3f}  (95% CI {stats_row['slope_ci_lo']:.3f},{stats_row['slope_ci_hi']:.3f})\n"
           f"intercept={stats_row['intercept']:.3f}  (95% CI {stats_row['intercept_ci_lo']:.3f},{stats_row['intercept_ci_hi']:.3f})\n"
           f"R²={stats_row['r_squared']:.3f}  r={stats_row['pearson_r']:.3f} (p={stats_row['pearson_p']:.1e})  "
           f"ρ={stats_row['spearman_rho']:.3f} (p={stats_row['spearman_p']:.1e})\n"
           f"RMSE={stats_row['rmse']:.3f}  MAE={stats_row['mae']:.3f}  bias(S2-S3)={stats_row['bias']:.3f}")
    at = AnchoredText(txt, loc=loc, prop=dict(size=9), frameon=True, borderpad=0.6, pad=0.4)
    at.patch.set_alpha(box_alpha)
    ax.add_artist(at)

def scatter_plot(x: np.ndarray, y: np.ndarray, s3_name: str, s2_name: str, stats_row: dict,
                 ax=None, figsize=(6,5), marker_size=15, alpha=0.6, stats_loc="lower right"):
    import matplotlib.pyplot as plt
    import numpy as np
    if ax is None:
        fig, ax = plt.subplots(figsize=figsize)

    ax.scatter(x, y, s=marker_size, alpha=alpha)

    # Deming fit line
    a, b = stats_row["intercept"], stats_row["slope"]
    xgrid = np.linspace(np.nanmin(x), np.nanmax(x), 200)
    yfit = a + b * xgrid
    ax.plot(xgrid, yfit, linewidth=2, label="Deming fit")

    # 1:1 dashed
    ax.plot(xgrid, xgrid, linestyle="--", linewidth=1, label="1:1")

    ax.legend(loc="best")

    xlabel = f"{s3_name}"
    ylabel = f"{s2_name}"
    if LOG10_TRANSFORM:
        xlabel += " (log10)"
        ylabel += " (log10)"
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    ax.set_title(f"{s2_name} vs {s3_name} (Deming)")

    add_stats_box(ax, stats_row, loc=stats_loc)
    return ax

def main():
    df = pd.read_csv(CSV_PATH)

    # coerce numeric-like columns
    for c in df.columns:
        if not pd.api.types.is_numeric_dtype(df[c]):
            df[c] = pd.to_numeric(df[c], errors='ignore')

    s3_col, s2_cols = (S3_COL_NAME, None) if S3_COL_NAME else (None, None)
    if s3_col is None or s2_cols is None:
        s3_col, s2_cols = autodetect_columns(df)

    df = df.drop_duplicates()
    s3_series = df[s3_col].astype(float)

    rows = []
    os.makedirs(PLOT_DIR, exist_ok=True)

    for s2_col in s2_cols:
        s2_series = df[s2_col].astype(float)
        xy = pd.DataFrame({s3_col: s3_series, s2_col: s2_series}).replace([np.inf, -np.inf], np.nan).dropna()
        if xy.empty or xy.shape[0] < 3:
            continue

        x = apply_transform(xy[s3_col].to_numpy())
        y = apply_transform(xy[s2_col].to_numpy())

        valid = np.isfinite(x) & np.isfinite(y)
        x, y = x[valid], y[valid]
        if len(x) < 3:
            continue

        x, y = remove_outliers_xy(x, y)
        if len(x) < 3:
            continue

        stats_row = regression_and_stats_deming(
            x, y, lam=LAMBDA_VAR_RATIO, n_boot=N_BOOT, seed=BOOT_SEED
        )
        stats_row.update(dict(s3_col=s3_col, s2_col=s2_col))

        # plot
        safe_name = re.sub(r"[^A-Za-z0-9_.-]+", "_", s2_col)
        out_plot = os.path.join(PLOT_DIR, f"s2_vs_s3_{safe_name}.png")
        fig, ax = plt.subplots(figsize=FIGSIZE)
        scatter_plot(x, y, s3_col, s2_col, stats_row, ax=ax, figsize=FIGSIZE,
                     marker_size=MARKER_SIZE, alpha=ALPHA, stats_loc="lower right")
        fig.tight_layout()
        fig.savefig(out_plot, dpi=200)
        plt.close(fig)

        rows.append(stats_row)

    if not rows:
        raise RuntimeError("No valid S2/S3 pairs found for Deming regression.")

    summary = pd.DataFrame(rows).sort_values(["r_squared", "pearson_r"], ascending=[False, False])
    summary_cols = [
        "s2_col", "s3_col", "n",
        "slope", "slope_se", "slope_ci_lo", "slope_ci_hi",
        "intercept", "intercept_se", "intercept_ci_lo", "intercept_ci_hi",
        "r_squared", "pearson_r", "pearson_p", "spearman_rho", "spearman_p",
        "rmse", "mae", "bias"
    ]
    summary = summary[summary_cols]
    summary.to_csv("s2_vs_s3_regression_summary.csv", index=False)

    print("✅ Done (Deming).")
    print(f"S3 column: {s3_col}")
    print(f"Compared S2 columns (n={len(summary)}):")
    for c in summary['s2_col']:
        print(f"  - {c}")
    print("\nWrote: s2_vs_s3_regression_summary.csv")
    print(f"Plots in: {os.path.abspath(PLOT_DIR)}/")

if __name__ == "__main__":
    main()


✅ Done (Deming).
S3 column: S3
Compared S2 columns (n=5):
  - S2_anchor1
  - s2_fp11
  - s2_fp21
  - s2_fp1
  - S2_orig1

Wrote: s2_vs_s3_regression_summary.csv
Plots in: /home/notebooks/ocean-colour/plots/
